# Clase 10: Desarrollo del Agente Virtual (Etapas 3 a 7)
En esta clase proyecto se trabajará desde la configuración del entorno hasta un prototipo funcional de un agente RAG (Retrieval-Augmented Generation) que responda consultas sobre documentos.

* ⚙️ Instalación de librerías y carga de los módulos de procesamiento.

* 📁 Subida de archivos PDF para contar con un corpus realista que el agente pueda consultar.

* 🧹 Limpieza de caracteres innecesarios y división del texto en fragmentos manejables. Facilita el análisis y evita errores por límites de tokens.

* 🧠 Conversión de los fragmentos en vectores numéricos (embeddings) utilizando un modelo preentrenado, para representar el significado semántico del contenido.

* 🗃️ Almacenamiento de los embeddings en un índice FAISS que permite realizar búsquedas rápidas por similitud semántica.

* 🔍 Codificación de las preguntas del usuario y recuperación del fragmento más relevante como contexto, utilizando el índice vectorial.

* 🤖 Generación de respuestas en lenguaje natural con el modelo, integrando el fragmento recuperado como base contextual (RAG).

#⚙️ Configuración del entorno
Configuramos el entorno de trabajo en Google Colab e instalamos las librerías necesarias para NLP, recuperación semántica y generación de texto.

In [ ]:
# Instalación de librerías necesarias
!pip install sentence-transformers -q
!pip install faiss-cpu -q
!pip install chromadb -q
!pip install pdfplumber -q
!pip install pandas -q
!pip install transformers -q
!pip install unstructured[local-inference] sentence-transformers faiss-cpu -q
!pip install transformers accelerate -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 65.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  G

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
import textwrap
import torch

modelo_nombre = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(modelo_nombre)
tokenizer.pad_token = tokenizer.eos_token
modelo_gpt = AutoModelForSeq2SeqLM.from_pretrained(modelo_nombre)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelo_gpt.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 1024)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 1024)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=1024, out_features=1024, bias=False)
              (k): Linear(in_features=1024, out_features=1024, bias=False)
              (v): Linear(in_features=1024, out_features=1024, bias=False)
              (o): Linear(in_features=1024, out_features=1024, bias=False)
              (relative_attention_bias): Embedding(32, 16)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=1024, out_features=2816, bias=False)
              (wi_1): Linear(in_features=1024, out_features=2816, bias=False)
       

# 📁 Bloque 2: Carga de datos
Montamos Google Drive y accedemos a la carpeta compartida para cargar automáticamente archivos en distintos formatos y almacenados para su análisis posterior, mientras que los pdf son procesados para extraer su contenido textual página por página. Una vez cargados, realizamos una limpieza básica del texto y lo dividimos en fragmentos manejables para poder trabajar con ellos en tareas de procesamiento de lenguaje natural.
> Coloquialmente se le dice “montar” porque, en informática, “montar” es hacer accesible un sistema de archivos externo como si fuera parte del sistema actual.


In [ ]:
# Paso 1: Montar Google Drive
from google.colab import drive
import os
import pandas as pd
import pdfplumber
import faiss
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Paso 2: Definir ruta a la carpeta
folder_path = '/content/drive/MyDrive/rocksubte'

# Paso 3: Listar todos los archivos de la carpeta
all_files = os.listdir(folder_path)

# Paso 4: Inicializar contenedores
excel_data = {}
pdf_text = {}

# Paso 5: Cargar archivos Excel
for file in all_files:
    if file.endswith('.xlsx'):
        file_path = os.path.join(folder_path, file)
        df = pd.read_excel(file_path)
        excel_data[file] = df
        print(f'📊 Excel cargado: {file} — {df.shape[0]} filas, {df.shape[1]} columnas')

# Paso 6: Cargar archivos PDF
for file in all_files:
    if file.endswith('.pdf'):
        file_path = os.path.join(folder_path, file)
        texto = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                texto += page.extract_text() + '\n'
        pdf_text[file] = texto
        print(f'📄 PDF cargado: {file} — {len(texto)} caracteres extraídos')

# Paso 7: Cargar archivo Google Sheets (si aplica)
# Solo si conoces el ID y el nombre de la hoja
sheet_id = 'TU_ID_DE_HOJA_DE_CALCULO'  # remplazar con el ID real
sheet_name = 'Nombre_de_la_hoja'       # remplazar con el nombre de la hoja

try:
    gsheet_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
    df_sheet = pd.read_csv(gsheet_url)
    print(f'🗂️ Google Sheet cargado — {df_sheet.shape[0]} filas, {df_sheet.shape[1]} columnas')
except:
    print("🔔 No se cargó ninguna hoja de cálculo de Google Sheets. Verifica el ID y nombre de la hoja si deseas usarlo.")

# Mostrar los nombres de archivos Excel cargados
print("📊 Archivos Excel cargados:")
for nombre in excel_data.keys():
    print(f" - {nombre}")

# Mostrar los nombres de archivos PDF cargados
print("\n📄 Archivos PDF cargados:")
for nombre in pdf_text.keys():
    print(f" - {nombre}")

📄 PDF cargado: revistalum.pdf — 48653 caracteres extraídos
🔔 No se cargó ninguna hoja de cálculo de Google Sheets. Verifica el ID y nombre de la hoja si deseas usarlo.
📊 Archivos Excel cargados:

📄 Archivos PDF cargados:
 - revistalum.pdf


## Lectura de archivos PDF
A continuación mostramos un resumen de los primeros 1000 caracteres de cada archivo PDF cargado desde la carpeta. Esto permite comprobar que el texto fue extraído correctamente y que el contenido está listo para procesarse en etapas posteriores como búsqueda semántica o generación de embeddings.

In [ ]:
# Mostrar vista previa de los primeros 1000 caracteres de cada PDF cargado
for nombre_archivo, texto in pdf_text.items():
    print(f"📄 Vista previa de: {nombre_archivo}")
    print(texto[:1000])
    print("-" * 80)  # separador visual


📄 Vista previa de: revistalum.pdf
ISSN: 2523-112X
D :
EsEnCuEnTROs DuRAnTE TIEmPOs vIOlEnTOs
r
El ock subTERRánEO y lA ulTRA IzquIERDA
80
sAnmARquInA En lOs
Wrangling during Violent Times: Punks and the Radical Left at San
Marcos University in the 80’s
FabIoLa bazo
RESUMEN
Este ensayo muestra cómo la radicalización política de los ochenta y sus expresiones de violencia
(que alcanzan su máxima expresión con el surgimiento de Sendero Luminoso) se manifestaron en
las identidades juveniles de ese periodo. Examina la interacción entre estudiantes sanmarquinos
de ultraizquierda y los rockeros subterráneos, ‘subtes’. Testimonios directos y la revisión de
publicaciones de ese periodo revelan cómo los prejuicios que primaban en la sociedad limeña de
entonces se reprodujeron en las interacciones de estos grupos juveniles, así como los límites de
los discursos contestatarios y de denuncia social de esa década, y la afirmación de identidades
antagónicas basadas en la otredad en ambos grupos.
Palab

In [ ]:
# Creamos una lista de fragmentos a partir de todo el texto extraído de los PDFs.
fragmentos = []
for texto in pdf_text.values():
    parrafos = texto.split("\n")  # Separamos el texto por saltos de línea para obtener párrafos

    for p in parrafos:
        if len(p.strip()) > 50: #🚧
            fragmentos.append(p.strip())

# Mostramos cuántos fragmentos resultaron útiles para análisis posterior
print(f"🧩 Total de fragmentos creados: {len(fragmentos)}")


🧩 Total de fragmentos creados: 333


#🚧 Bloque 3: Limpieza y segmentación
Preparamos el texto para su procesamiento, limpiando y dividiéndolo en fragmentos manejables (chunking).

## Limpieza
Antes de trabajar con documentos en lenguaje natural, es necesario realizar una etapa de limpieza. Los textos extraídos desde archivos PDF o escaneos digitales suelen contener saltos de línea, espacios innecesarios, símbolos invisibles y caracteres de control que interfieren en el procesamiento posterior. La limpieza busca estandarizar el contenido textual eliminando elementos irrelevantes sin alterar el significado, permitiendo que los modelos de procesamiento de lenguaje (NLP) interpreten correctamente la información. Además, una buena limpieza mejora la calidad de los embeddings y reduce el ruido durante la búsqueda semántica o la generación de respuestas.

In [ ]:
import re
import unicodedata

def limpiar_texto(texto, minusculas=True, eliminar_puntuacion=False):

    texto = unicodedata.normalize('NFKC', texto)# Normalizar el texto (conservando tildes y ñ)
    texto = texto.replace('\xa0', ' ').replace('\n', ' ').replace('\r', ' ')# Reemplazar saltos y caracteres no imprimibles
    texto = re.sub(r'\s+', ' ', texto).strip() # Eliminar caracteres de control y múltiples espacios
    # Convertir a minúsculas si se indica
    if minusculas:
        texto = texto.lower()
    # Eliminar puntuación (excepto letras acentuadas y ñ) si se desea
    if eliminar_puntuacion:
        texto = re.sub(r'[^\w\sáéíóúüñÁÉÍÓÚÜÑ]', '', texto)
    return texto
texto_limpio = limpiar_texto(pdf_text['revistalum.pdf'])

## Chunking

Creamos una función para dividir el texto completo en bloques de N palabras, lo que permite alimentar a los modelos de lenguaje sin pasarnos del límite que pueden procesar a la vez.

In [ ]:
# Fragmentación en bloques de 300 palabras
def dividir_en_fragmentos(texto, tamano=300):#🚧
    palabras = texto.split()
    fragmentos = []
    for i in range(0, len(palabras), tamano):
        fragmento = " ".join(palabras[i:i+tamano])
        fragmentos.append(fragmento)
    return fragmentos

fragmentos = dividir_en_fragmentos(texto_limpio)
print(f"Cantidad de fragmentos: {len(fragmentos)}")
print(fragmentos[2])

Cantidad de fragmentos: 26
“uno de los peruana” (p.110), sino también recalcó que la problemas más importantes en la sociedad pe- radicalización de los jóvenes de los ochenta ruana contemporánea” (manrique, 2002, p.144). “parecería proclive a sendero luminoso” ya debido a la violencia política, en menos de tres que esta juventud popular no expresaba su ra- años el gobierno elegido democráticamente del dicalización a través de canales institucionales presidente fernando belaúnde (1980-1985) como los partidos políticos, como lo hicieron autorizaba la paulatina militarización del país los jóvenes de las décadas de los sesenta y y la suspensión de derechos y garantías ciu- setenta. más bien, negaba toda validez al régi- dadanas. su sucesor, alan garcía (1985-1990), men político e institucional porque este carecía prometió una estrategia antisubversiva dis- de la capacidad de construir canales partici- tinta que no delegaría el control a las fuerzas pativos para una juventud que enfrentaba 

## parse_pdf()

#👾 Bloque 4: Embeddings semánticos
Este modelo convierte cada fragmento de texto en un vector numérico en un espacio semántico. Así, fragmentos similares estarán más cerca entre sí, lo que es ideal para tareas de recuperación de información (RAG, QA, etc.).

El modelo funciona como un "traductor" que lleva las frases al idioma de las matemáticas, permitiendo comparar significados en vez de solo palabras.

In [ ]:
# Cargar el modelo de embeddings
from sentence_transformers import SentenceTransformer
modelo = SentenceTransformer('all-MiniLM-L6-v2') #🚧

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Codificar los fragmentos en vectores
vectores = modelo.encode(fragmentos)

El método *.encode()* toma como entrada una lista de textos (en este caso la que llamamos "fragmentos" ) y devuelve una lista de vectores —uno por cada fragmento— donde cada vector es una representación multidimensional del contenido semántico del texto.

+ En el modelo es all-MiniLM-L6-v2, cada vector tendrá 384 dimensiones, y cada valor en ese vector representa una característica semántica aprendida durante el entrenamiento del modelo.

+ Estos vectores no codifican las palabras en sí, sino el significado del fragmento completo, permitiendo que textos distintos pero con significados similares queden cerca entre sí en ese espacio vectorial. Este proceso de codificación transforma un texto en lenguaje natural en matrices numéricas  que los algoritmos pueden operar directamente.

+ Una vez que se tienen los vectores, se pueden hacer operaciones como calcular distancias, similitudes, agrupamientos o búsquedas semánticas.

# 🗃️ Bloque 5: Indexación
Esta parte es clave para un sistema RAG (Retrieval-Augmented Generation): estamos construyendo el motor que buscará los fragmentos más relevantes para una consulta.

>FAISS es como un "Google interno", pero en lugar de buscar por palabras, busca por significados representados como vectores.`

In [ ]:
# Creamos un índice de búsqueda usando FAISS, una librería optimizada para búsquedas vectoriales rápidas.

# Cargamos un modelo de embeddings multilingüe que genera vectores para cada fragmento de texto.
modelo = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')  # 🚧   Puedes cambiar el modelo por otro más rápido o más preciso
vectores = modelo.encode(fragmentos) # Convertimos cada fragmento en su correspondiente vector numérico.

# Creamos el índice vectorial FAISS usando distancia L2 (euclidiana).
indice = faiss.IndexFlatL2(vectores.shape[1])  # 🚧  También puedes usar 'IndexFlatIP' si prefieres similitud por producto punto
indice.add(vectores) # Añadimos los vectores al índice
print(f"🔍 Fragmentos indexados en FAISS: {indice.ntotal}")


🔍 Fragmentos indexados en FAISS: 26


# 🔍 Bloque 6: Búsqueda semántica

Esta función es el centrto del sistema de recuperación. Recibe una pregunta, la convierte en vector y busca en el índice FAISS los fragmentos más parecidos semánticamente.

In [ ]:
# Definimos una función para recuperar los fragmentos más relevantes para una pregunta dada.

def buscar_fragmentos_relevantes(pregunta, k=5):  # 🚧  'k' indica cuántos fragmentos similares queremos recuperar
    # Convertimos la pregunta en un vector usando el mismo modelo de embeddings
    vector_pregunta = modelo.encode([pregunta])

    # Buscamos los 'k' fragmentos más cercanos en el espacio vectorial
    distancias, indices = indice.search(vector_pregunta, k=k)

    # Recuperamos los fragmentos correspondientes a los índices encontrados
    fragmentos_recuperados = [fragmentos[i] for i in indices[0]]

    # Devolvemos todos los fragmentos concatenados como un solo texto
    return " ".join(fragmentos_recuperados)


Esta función toma el prompt (la pregunta + contexto recuperado) y genera una respuesta textual con el modelo T5. Es la fase de "generación aumentada" del enfoque RAG: ya que el modelo responde usando el contexto más relevante.

In [ ]:
# Esta función genera una respuesta a partir del prompt (pregunta + contexto), usando el modelo local cargado previamente.

def modelo_gpt_generate(prompt, max_length=200):  # 🚧  Puedes ajustar la longitud máxima de la respuesta generada
    # Tokenizamos el prompt y lo convertimos a tensores para el modelo
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to(device)

    # Generamos texto a partir del input tokenizado
    outputs = modelo_gpt.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_length,                  # 🚧  Limita cuántos tokens nuevos puede generar el modelo
        pad_token_id=tokenizer.pad_token_id,
        do_sample=True,                             # 🚧  Activa muestreo en lugar de greedy decoding
        # temperature=0.5,                          # (Desactivado por ahora) Más alto = más impredecible
        # top_p=0.9                                 # (Desactivado por ahora) Usa solo los tokens más probables
    )

    # Decodificamos la respuesta del modelo (removiendo tokens especiales)
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return respuesta.strip()



# 🤖 Bloque 7: Retrieval-Augmented Generation (RAG)



In [ ]:
#Cargar modelo generativo
rag_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
rag_tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
rag_pipeline = pipeline("text2text-generation", model=rag_model, tokenizer=rag_tokenizer)

Device set to use cuda:0


In [ ]:
# 🧠 Agente virtual con recuperación y generación
def agente_virtual(pregunta):
    contexto = buscar_fragmentos_relevantes(pregunta)

    # 🔧 Limita la longitud del contexto si es demasiado largo
    if len(contexto) > 1300:
        contexto = contexto[:1300]

    # Armamos el prompt con el contexto y la pregunta
    prompt = f"Responde la siguiente pregunta usando el contexto:\n{contexto}\n\nPregunta: {pregunta}"

    # Generamos la respuesta usando el modelo
    respuesta = modelo_gpt_generate(prompt)

    return respuesta, contexto

# ✅ Importamos textwrap para formatear la salida del texto


# Pregunta de prueba
pregunta = "¿En que contexto sociopolitico nació el rock subte?"

# Ejecutamos el agente virtual
respuesta, contexto = agente_virtual(pregunta)

# Mostramos parte del contexto recuperado (hasta 1000 caracteres)
print("📄 Contexto recuperado:")
print(textwrap.fill(contexto[:1000], width=100))  # Formatea el texto para mejor lectura

# Mostramos la respuesta del agente
print("\n📢 Respuesta del agente virtual:\n")
print(textwrap.fill(respuesta, width=100))



📄 Contexto recuperado:
contestatarios y de literal de textos ideológicos fundacionales denuncia social de los años ochenta.
asociados al marxismo y al maoísmo), prove- nientes de sectores populares y de familias el rock
subterráneo migrantes de primera generación. por el otro, el punk rock, su “actitud” y el “hazlo-tú-
mis- los jóvenes que participaban en la escena del mo” detonarían en lima en 1985 –casi diez rock
subterráneo, en su mayoría provenientes años después que en londres, y el mismo de familias urbanas,
sin un proyecto ideoló- año que alan garcía fue elegido presidente gico u organizativo, y que desde
una posición por primera vez– con cinco grupos funda- individualista rechazaban también el régimen
cionales: leusemia (1983), narcosis (1984), político e institucional a través de su música. autopsia
(1985), guerrilla urbana (1985) y ambos grupos se enfrascaron en una discu- zcuela cerrada (1985).
los subtes escribían, sión sobre la autenticidad de sus posiciones, producían, di